In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
train = pd.read_csv("Data/ailsa/Ailsadata2.csv", index_col=0)
test = pd.read_csv("Data/ailsa/Ailsatest2.csv")

/usr/local/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3251: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [4]:
train_cols = train.columns.to_list()
test_cols = test.columns.to_list()

In [5]:
train.head()

,TransactionID,isFraud,TransactionAmt,ProductCD,card3,card4,card6,addr2,dist1,P_emaildomain,...,M1,M2,M3,M4,M5,M6,M7,M8,M9,hour
1,2987000,0,68.5,W,150,discover,credit,87,19.000000,google,...,1,1,1,M2,0,1,0,0,1,0
2,2987001,0,29.0,W,150,mastercard,credit,87,118.502948,google,...,1,1,1,M0,1,1,0,0,1,0
3,2987002,0,59.0,W,150,visa,debit,87,287.000000,Microsoft,...,1,1,1,M0,0,0,0,0,0,0
4,2987003,0,50.0,W,150,mastercard,debit,87,118.502948,yahoo,...,1,1,1,M0,1,0,0,0,1,0
5,2987004,0,50.0,H,150,mastercard,credit,87,118.502948,google,...,1,1,1,M0,0,0,0,0,1,0


In [6]:
from sklearn.ensemble import RandomForestClassifier

forest = RandomForestClassifier(n_estimators=100, bootstrap=False)


In [7]:
y = train.isFraud

num_cols = test.select_dtypes(exclude=object).columns.to_list()

X = train[num_cols]

forest.fit(X,y)

RandomForestClassifier(bootstrap=False)

In [8]:
X_test = test[num_cols]

X_test.head()

,TransactionID,TransactionAmt,dist1,C1,C2,C3,C4,C5,C6,C7,...,D15,M1,M2,M3,M5,M6,M7,M8,M9,hour
0,3663549,31.95,1.0,6.0,6.0,0.0,0.0,3.0,4.0,0.0,...,409.0,1,1,0,0,0,1,1,1,0
1,3663550,49.00,4.0,3.0,2.0,0.0,0.0,0.0,1.0,0.0,...,634.0,1,0,0,0,0,0,0,1,0
2,3663551,171.00,2635.0,2.0,2.0,0.0,0.0,0.0,5.0,0.0,...,97.0,1,1,0,0,0,0,0,0,0
3,3663552,284.95,17.0,5.0,2.0,0.0,0.0,1.0,1.0,0.0,...,242.0,1,1,1,0,1,0,0,1,0
4,3663553,67.95,6.0,6.0,6.0,0.0,0.0,2.0,5.0,0.0,...,22.0,1,1,1,0,0,0,1,1,0


In [9]:
X_test_fill = X_test.fillna(X_test.mean())

X_test_fill.head()

,TransactionID,TransactionAmt,dist1,C1,C2,C3,C4,C5,C6,C7,...,D15,M1,M2,M3,M5,M6,M7,M8,M9,hour
0,3663549,31.95,1.0,6.0,6.0,0.0,0.0,3.0,4.0,0.0,...,409.0,1,1,0,0,0,1,1,1,0
1,3663550,49.00,4.0,3.0,2.0,0.0,0.0,0.0,1.0,0.0,...,634.0,1,0,0,0,0,0,0,1,0
2,3663551,171.00,2635.0,2.0,2.0,0.0,0.0,0.0,5.0,0.0,...,97.0,1,1,0,0,0,0,0,0,0
3,3663552,284.95,17.0,5.0,2.0,0.0,0.0,1.0,1.0,0.0,...,242.0,1,1,1,0,1,0,0,1,0
4,3663553,67.95,6.0,6.0,6.0,0.0,0.0,2.0,5.0,0.0,...,22.0,1,1,1,0,0,0,1,1,0


In [10]:
predictions = forest.predict(X_test_fill)

In [11]:
output = pd.DataFrame({'TransactionID': test.TransactionID,
                       'isFraud': predictions})

output.to_csv("Data/ailsa/testing.csv", index=False)

actually ok wtf

now try with categorical columns

In [12]:
new_train = train.drop('isFraud', axis=1)

new_train = pd.get_dummies(new_train)

In [13]:
new_test = test.copy()

new_test[num_cols]= X_test_fill

cat_cols = new_test.select_dtypes(include=object).columns.to_list()

new_test[cat_cols].fillna(new_test[cat_cols].mode())

new_test = pd.get_dummies(new_test)

In [14]:
new_train = new_train.drop('card6_debit or credit', axis=1) # needed so both test and train have smae cols

In [15]:
forest_2 = RandomForestClassifier()

forest_2.fit(new_train, y)

RandomForestClassifier()

In [16]:
new_preds = forest_2.predict(new_test)

/usr/local/lib/python3.9/site-packages/sklearn/base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names must be in the same order as they were in fit.

  warnings.warn(message, FutureWarning)


ValueError: X has 80 features, but RandomForestClassifier is expecting 81 features as input.

In [ ]:
output = pd.DataFrame({'TransactionID': test.TransactionID,
                       'isFraud': new_preds})

#output.to_csv("Data/ailsa/testing2.csv", index=False)

In [ ]:
L = len(new_train)

In [ ]:
subset = new_train[:int(L)].copy()

sub_train = subset[:int(len(subset)*0.8)]
sub_test = subset[int(len(subset)*0.8):]

In [ ]:
new_for = RandomForestClassifier(n_estimators=100, max_depth=10)

In [ ]:
sub_y_train = sub_train['isFraud']

sub_x_train = sub_train.drop('isFraud', axis=1)

print(sub_x_train.columns.to_list())

new_for.fit(sub_x_train, sub_y_train)

In [ ]:
sub_y_test = sub_test['isFraud']

sub_x_test = sub_test.drop('isFraud', axis=1)

print(sub_x_test.columns.to_list())

sub_y_preds = new_for.predict(sub_x_test)

In [ ]:
from sklearn.metrics import roc_auc_score, confusion_matrix, f1_score

for i in range(2, 10, 1):
    clf = RandomForestClassifier(criterion='gini', max_depth=30, bootstrap=False, max_features=7, min_samples_leaf=1,
                                 min_samples_split=4,)

    clf.fit(sub_x_train, sub_y_train)

    p = clf.predict(sub_x_test)

    print("Features: ", i)
    print()
    print(confusion_matrix(sub_y_test, p))
    print()
    print(f1_score(sub_y_test,p))
    print()
    print(roc_auc_score(sub_y_test, clf.predict_proba(sub_x_test)[:, 1]))
    print("-------------------------------------")